<a href="https://colab.research.google.com/github/andinam77/DMA_LAB2/blob/master/PERCEPTOR.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from sklearn import preprocessing
import numpy as np
import scipy.linalg.blas as FB
import math 
import pandas as pd

import numba as nb
from tqdm import tqdm


from numpy.linalg import multi_dot
super_dot = lambda v, w: FB.dgemm(alpha=1., a=v.T, b=w.T, trans_b=True)

In [2]:
#@nb.vectorize([nb.float32(nb.int32, nb.int32, nb.int32, nb.float32, nb.float32, nb.float32)])
@nb.jit #(nopython=True)
def RBF_kernel(q1, q2,tr,sigma,TR,TEt):
    
    '''
    TEt=TE
    sigma=0.01
    q1=686
    q2=1500
    tr=1
    '''
    #global XX
    #global XXt
    
    #XXt==np.array(XXt.drop('index',axis=1))
    #XX=np.array(XX.drop('index',axis=1))
    #XXt==np.array(XXt)
    #XX=np.array(XX)
            
    if tr == 1:

        #s0= sum([XX[q1][i]*XX[q2][i] for i in range(len(XX[q1]))])
        
        s0=np.dot(np.asarray(TR[q1], order='C'),np.asarray(TR[q2], order='C'))
        #s0=np.dot(XX[q1], XX[q2])
        #s0 = multi_dot(XX[1].T, XX[3])
        s1 = s0 * -2
        dot11 = np.dot(np.asarray(TR[q1], order='C'),np.asarray(TR[q1], order='C'))
        #dot11=np.dot(XX[q1],XX[q1])
        #dot11=sum([XX[q1][i]*XX[q1][i] for i in range(len(XX[q1]))])
        dot22 = np.dot(np.asarray(TR[q2], order='C'), np.asarray(TR[q2], order='C'))
        #dot22=np.dot(XX[q2], XX[q2])
        #dot22=sum([XX[q2][i]*XX[q2][i] for i in range(len(XX[q1]))])
        
    
       
    else:

        #s0= sum([XX[q1][i]*XXt[q2][i] for i in range(len(TEt[q1]))])
        s0 = np.dot(np.asarray(TR[q1], order='C'), np.asarray(TEt[q2], order='C'))
        s1 = s0 * -2
        dot11 =np.dot(np.asarray(TR[q1], order='C'), np.asarray(TR[q1], order='C'))
        #dot11=sum([XX[q1][i]*XX[q1][i] for i in range(len(XX[q1]))])

        dot22 = np.dot(np.asarray(TEt[q2], order='C'), np.asarray(TEt[q2], order='C'))
        #dot22=sum([XXt[q2][i]*XXt[q2][i] for i in range(len(XXt[q1]))])
  
    s1 = s1 + dot11 + dot22
    if (-s1 / sigma) <100:
        RBF_kernel_ = math.exp(-s1 / sigma) 
    else:
        RBF_kernel_ = -999999
         
    #print(RBF_kernel_,' q1 q2 ',s0,' 2*q1 y q2 ',s1,'q1 q1  ',dot11,'  q2 q2 ', dot22)
           
        
    return RBF_kernel_